In [ ]:
%run Utils.ipynb

In [ ]:
def trace_calculator(a: int, trance_len: int) -> list:
    trace = [1, a]
    for i in range(2,trance_len):
        trace.append((trace[i-1] + trace[i-2])%PRIME)
    return trace

In [5]:
def polynomial_evaluation (trace: list, generator: IntegerMod_gmp) -> Polynomial_zmod_flint:
    points =[]
    for i, y in enumerate (trace): 
        points.append((generator**i, y))
    
    R = FIELD['x']
    polynomial = R.lagrange_polynomial(points)
    return polynomial

In [7]:
def compositon_polynomial(poly_list: list, random_co: dict):
    cp=0*x
    for poly in poly_list:
        cp=cp + poly * random_co[poly]
    return cp


In [8]:
def constrains_polynomials(poly: Polynomial_zmod_flint, trance_len: int, Y: IntegerMod_gmp, index_y: int , gen: IntegerMod_gmp):
    n=trance_len
    p1 = (poly-1)._divide_if_possible(x-gen**0)
    p2 = (poly - Y)._divide_if_possible(x-gen**index_y)
    
    # (x-g**0)(x-g**1)...(x-g**(n-1)) = x**n-1
    # 
    constrain_3_numer = poly(gen ** 2 * x) - poly(gen * x) - poly(x)
    # constrain_3_numer should divide by all powers of gen: x=g**0, .... x= g**(n-3)
    constrain_3_denom = (x**n-1)._divide_if_possible( (x-gen**(n-1)) * (x-gen**(n-2)) )
    # p3 = (poly(gen ** 2 * x) - poly(gen * x) - poly(x))*(x-gen**(n-1)) \
    #     * (x-gen**(n-2))*(x-gen**(n-3))/(x**n-1)
    p3 = constrain_3_numer._divide_if_possible(constrain_3_denom)
    return (p1, p2, p3)

In [15]:
def domain_extension( trace_len: int, group_gen: IntegerMod_gmp, field_gen: IntegerMod_gmp):
    coset_set = [field_gen*(group_gen**i) for i in range (trace_len)]
    new_coset_set = [coset_set[reverse_bit(i, log(trace_len,2))] for i in range (trace_len)]
    return new_coset_set

In [17]:
def fri(poly: Polynomial_zmod_flint, domain: list , degree = 1024, queries = 8) -> dict:
    
    domain_length = len(domain)
    index_to_sample = [0]*queries
    proof = {} #{stage: {'root': merkle root, 'pathes':[pathes],'random':random number for naxt-stage}}
    for stage in range (log(degree, 2)):
        
        pathes = [] # [(value, path),...]
        merkel_root, merkle_layer = commit(poly, domain)
        #first stage: evaluation above the whole domain and creation of merkle tree
        if stage == 0: #qurie at queries/2 random indexes
            for i in range(0,queries,2):
                index_to_sample[i] = fiat_shamir_random(merkel_root,i)%domain_length
                index_to_sample[i+1] = index_to_sample[i]+(-1)**(index_to_sample[i]%2)
        else: #in every other stage - brings the indexes of the negative element in the domain
            new_index_to_sample = []
            for index in index_to_sample:
                index//=2
                temp_index = index + (-1)**(index%2)
                if temp_index not in new_index_to_sample:
                    new_index_to_sample.append(temp_index)
            index_to_sample = new_index_to_sample
        for index in index_to_sample:
            value_at_index, path = evaluate_points_and_path(merkle_layer, int(index))
            pathes.append((value_at_index, path))
        #second stage: takes n number randoms, and claculate n/2 time P(x_i) 1<i<n/2 + merkle path for them

        rand = fiat_shamir_random(merkel_root)
        proof[stage]= {'root':merkel_root ,'pathes':pathes,'random': rand}
        poly, domain = fri_next_layer(poly=poly, domain=domain, rand = rand)
        #third stage: calculates FRI next Layer
    
    return proof

In [18]:
def commit(poly: Polynomial_zmod_flint, domain: list):
    #first stage: evaluation above the whole domain and creation of merkle tree
    points=[(d, poly(d)) for d in domain]
    tree = MerkeTree(domain=points)
    return (tree.root, tree)

In [21]:
def fri_next_layer(poly: Polynomial_zmod_flint, domain: list , rand: int):
    #calculate the polynomial and the domain of the next stage
    even = 0*x
    odd = 0*x
    for degree,coef in poly.dict().items():
        if degree%2==0:
            even = even + coef*x**(degree//2)
        else:
            odd = odd + coef*x**(degree//2)
    next_layer = even + rand*odd
    new_domain = []
    for i in range(0,len(domain),2):
        assert domain[i]**2 == domain[i+1]**2
        new_domain.append(domain[i]**2)
    return next_layer, new_domain

In [38]:
def evaluate_points_and_path(tree: MerkeTree, index: int):
    return tree.get_value_and_path_by_index(index=index)

In [41]:
def prove(a: int, trace_length: int, destination, queries: int = 8):
    trace = trace_calculator(a,trace_length)
    Y = trace[destination]
    poly = polynomial_evaluation(trace, gen1024)
    p1, p2, p3 = constrains_polynomials(poly, len(trace), Y, destination, gen1024)
    domain_size = len(trace)*queries
    domain_gen = field_gen ** ((PRIME-1)/domain_size)
    domain = domain_extension(domain_size , domain_gen, field_gen)
    proof_stage_one = {} #{name:{'root' = root, 'value' = value,'path' = path}}
    random_co = {}
    merkle_p = {}
    root_p = {}
    value_at_index= {}
    path = {}
    for p in [p1, p2, p3]:
        root_p[p], merkle_p[p] = commit(p, domain)
        random_co[p] = fiat_shamir_random(root_p[p])
        
    cp = compositon_polynomial([p1, p2, p3], random_co)
    root_p[cp], merkle_p[cp] = commit(cp, domain)
    index = fiat_shamir_random(root_p[cp])%len(domain)
    polynomials = [p1, p2, p3, cp]
    names = ['p1', 'p2', 'p3', 'cp']
    for p , name in zip(polynomials, names):
        value_at_index[p], path[p] = evaluate_points_and_path(merkle_p[p], index)
        internal_proof_dict={}
        internal_proof_dict['root'] = root_p[p]
        internal_proof_dict['value'] = value_at_index[p]
        internal_proof_dict['path'] = path[p]
        proof_stage_one[name] = internal_proof_dict
    
#   return proof_stage_one
    proof_stage_two = fri(cp, domain , degree = 1024, queries = 8)
    #{stage: {'root': merkle root, 'pathes':[pathes],'random':random number for naxt-stage}}
    
    return proof_stage_one, proof_stage_two

In [ ]:
# poly(x)
# poly(gen ** 2 * x) 
# poly(gen * x) 